# OR107

In [4]:
import numpy as np
import pandas as pd 
from datetime import datetime

### 讀取資料

In [14]:
grades = pd.read_csv('../../data/OR107 grades.csv')
v_info = pd.read_excel('../../data/OR107 videos_sep.xlsx', sheet_name = 'video info')
records = pd.read_excel('../../data/OR107 videos_sep.xlsx', sheet_name = 'watching records')
student_id = pd.read_excel('../../data/OR107 videos_sep.xlsx', sheet_name = 'student id')

### 其中、期末成績

In [21]:
# 開一個 table 存等一下計算的資料
student_info = pd.DataFrame(data = grades.drop(0)['student_id'])
student_info['project'] = grades['Final project (2126)']
student_info['final'] = grades['Final Exam (2231)']

### 觀看時間、觀看速度、平均觀看次數

In [24]:
# 計算總觀看時間
total_spent_time = [] # 總觀看時間
total_watch_video_time = [] # 觀看的總影片時間

for i in student_info['student_id']:
    watch_record_i = records[records['student_id'] == int(i)]
    # 抓出有在看的資料
    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]
    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
    total_watch_video_time.append(sum(video_time_i))
    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

student_info['total_watch_time'] = total_spent_time

# 計算撥放速度
total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
student_info['avg_playback_rate'] = avg_playback_rate

# 計算影片平均觀看次數
length_l = list(v_info['meta'])# length of each lecture
length_l = [int(i.split('n=>')[1].split('}')[0]) for i in length_l]
total_length_l = sum(length_l)
student_info['avg_freq'] = student_info['total_watch_time']/(total_length_l/student_info['avg_playback_rate'])


student_info

,student_id,project,final,total_watch_time,avg_playback_rate,avg_freq
1,88,NaN,NaN,12633.775000,1.161569,0.212223
2,89,92.69,41.0,38527.933333,1.400646,0.780402
3,90,NaN,NaN,0.000000,0.000000,0.000000
4,91,NaN,NaN,19812.533333,1.607265,0.460513
5,92,NaN,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
79,166,94.76,70.0,3979.116667,1.686555,0.097051
80,167,NaN,NaN,5726.700000,1.328165,0.109994
81,168,94.61,75.0,29035.216667,1.499558,0.629655
82,169,98.50,82.0,31742.950000,1.417638,0.650769


### Save data

In [29]:
student_info.to_excel('../../data/許哲/OR107.xlsx')

In [28]:
import statsmodels.formula.api as smf
result = smf.ols('final ~ avg_playback_rate', data = student_info).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  final   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9342
Date:                Wed, 27 May 2020   Prob (F-statistic):              0.338
Time:                        00:38:51   Log-Likelihood:                -217.89
No. Observations:                  52   AIC:                             439.8
Df Residuals:                      50   BIC:                             443.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            57.9626     13.79